1. What are the possible scenarios in which the data could be used?
2. Use one of these scenarios to develop a model of your choice, using the tools of your choice.
    * Please describe how you would engage with the business to clarify requirements.
    * Show thinking behind setting up data model and how this is designed.
    * Prepare a presentation of your findings to present to the business stakeholders


# Customer Intelligence

Customer intelligence is a quite a large domain, and is studied extensively in both industry and academia. The whole field in general can often find one need at it's inception.

How can we understand our customer better?

Often Data Science teams are faced with this question when working in industry, and many teams tackle the problem in a wide variety of ways. However, it is important to note that this question is almost impossible to practically answer due to two reasons.

* What does *better* mean?
What metric tells you better? Is it measurable? Is the *better* consistent throughout the organisation? Is it about sales? Perhaps it's about how your brand is perceived? Or maybe it's about developing more suitable products?
* Better is a rolling goalpost.
Understanding customers better is always a moving target. You can always understand better, you can always look at the customer cake in different angles.

Enought theorising, let's answer some questions.

## 1. What are the possible scenarios in which the data could be used?

Even without diving into the data, we can generate scenarios which this data may come in handy. Let's discuss some examples how this data may be used to generate business value

### Business intelligence

This type of data, is most often used for development of BI metrics. The primary focus of BI is to give a better situational awareness for operational teams. Much of the data in question here would be usable for all levels of the business. For example, sales metrics are important at the highest level of business to determine business strategies. 

Business intelligence should give an instant snapshot into the current state of your business. Typically these are expressed as summary statistics via visualisations.

In [1]:
import os
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from datetime import datetime
import plotly.graph_objs as go
from scipy import stats

init_notebook_mode(connected=True)

In [2]:
city = pd.read_csv("data/city.csv", names = ['CityID', 'CityName', 'Zipcode', 'CountryID'], header = 0)
country = pd.read_csv("data/country.csv" , names = ['CountryID', 'CountryName', 'CountryCode'], header = 0)
customer = pd.read_csv("data/customer.csv", names = ['CustomerID', 'FirstName', 'MiddleInitial', 'LastName',
       'CityID', 'Address'], header = 0)
product = pd.read_csv("data/product.csv", names = ['ProductID', 'ProductName', 'Price', 'CategoryID', 'Class',
       'ModifyDate', 'Resistant', 'IsAllergic', 'VitalityDays'], header = 0)
product_category = pd.read_csv("data/product_category.csv", names = ['CategoryID', 'CategoryName'], header = 0)
staff = pd.read_csv("data/staff.csv", names = ['EmployeeID', 'FirstName', 'MiddleInitial', 'LastName',
       'BirthDate', 'Gender', 'CityID', 'HireDate'], header = 0)
transaction = pd.read_csv("data/transaction.csv", names = ['SalesID', 'SalesPersonID', 'CustomerID', 'ProductID',
       'Quantity', 'Discount', 'TotalPrice', 'SalesDate',
       'TransactionNumber'], header = 0)

In [3]:
transaction.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,NaN,"0,00",2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,NaN,"0,00",2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,NaN,"0,00",2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,"0,00",2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,NaN,"0,00",2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3


In [4]:
# Convert type and then to date
transaction['SalesDate'] = pd.to_datetime(transaction['SalesDate'], errors = 'coerce')
transaction['Date'] =  transaction['SalesDate'].dt.date
transaction['DayOfWeek'] =  pd.to_datetime(transaction['Date']).dt.weekday
transaction['Day'] =  pd.to_datetime(transaction['Date']).dt.day_name()

In [5]:
transaction.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,Date,DayOfWeek,Day
0,1,6,27039,381,7,NaN,"0,00",2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,2018-02-05,0.0,Monday
1,2,16,25011,61,7,NaN,"0,00",2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,2018-02-02,4.0,Friday
2,3,13,94024,23,24,NaN,"0,00",2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,2018-05-03,3.0,Thursday
3,4,8,73966,176,19,0.2,"0,00",2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,2018-04-07,5.0,Saturday
4,5,10,32653,310,9,NaN,"0,00",2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,2018-02-12,0.0,Monday


In [6]:
# Aggregate sales by day
tx_by_day = transaction.groupby(['Date']).count().reset_index()
# Day of week
tx_by_day['Date'] = pd.to_datetime(tx_by_day['Date'])
tx_by_day.head()

,Date,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber,DayOfWeek,Day
0,2018-01-01,7755,7755,7755,7755,7755,1435,7755,7755,7755,7755,7755
1,2018-01-02,7674,7674,7674,7674,7674,1570,7674,7674,7674,7674,7674
2,2018-01-03,7575,7575,7575,7575,7575,1526,7575,7575,7575,7575,7575
3,2018-01-04,7646,7646,7646,7646,7646,1455,7646,7646,7646,7646,7646
4,2018-01-05,7876,7876,7876,7876,7876,1510,7876,7876,7876,7876,7876


In [7]:
# Plot time series.
iplot({'data':[go.Scatter(x = tx_by_day['Date'], y = tx_by_day['SalesID'])] , 'layout': {'title':'Sales',}} )

### QA and Analytics

Question answering usually using descriptive analytics is the most common usage of date warehouses. After all they're placed in OLAP databases so you can do, well, **analytics**. From my previous experience being in the "data science" teams of non-data businesses, QA analytics seems to get the most attention. 

Analytics takes BI one step further. While BI is about seeing the state of your business at any given time, analytics is about helping the decision making process. Let's see what some of the questions that may arise in a e-commerce retailer and how the given data may be used to aid the decision making process.

#### Scenario: Add campaign
Add campaigns have become both easier and harder to carry out. They are easier in the sense that the delivery medium has been somewhat started to become dominated by social media platforms which has been the traditional how to split the budget has been less important. Regardless, post campaign analysis can be done using sales and product data.

#### Scenario: Product development
Product development can benefit significantly from Analytics pieces around transactions. Typically at the start of a development lifecycle one can ask question as following. 

1. What are our best selling products?
2. What are our worst selling products?

Ignoring market forces for now, a product development manager may use the answers to these questions to decide if it's better to invest in improving a low performing product or provide a product with more than one function. (Condition vs Shampoo + conditioner)

Let's derive some visualisations / items for these items.

In [24]:
qt_pid = transaction.groupby(['ProductID'])['Quantity'].sum().reset_index().sort_values(by= 'Quantity', ascending = False)
qt_pid.head()

,ProductID,Quantity
360,361,31116
326,327,30811
164,165,30405
424,425,30365
60,61,30319


In [25]:
pd_dets = qt_pid.merge(product,on='ProductID',sort=False).merge(product_category,on = 'CategoryID',sort = False)
pd_dets.head()

,ProductID,Quantity,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays,CategoryName
0,361,31116,Coffee Decaf Colombian,"36,4516",2,Medium,2017-10-20 11:54:22.960,Weak,True,71.0,Shell fish
1,327,30811,"Appetizer - Mini Egg Roll, Shrimp","7,1436",2,Medium,2017-06-20 08:14:28.810,NaN,False,NaN,Shell fish
2,357,30057,Tilapia - Fillets,"6,8332",2,Medium,2017-05-27 00:14:44.530,Durable,False,NaN,Shell fish
3,60,30007,"Pepper - Paprika, Hungarian","31,6949",2,Medium,2017-09-29 18:42:08.470,Weak,NaN,NaN,Shell fish
4,430,29900,Milk - 1%,"46,2589",2,Low,2017-05-12 04:20:35.330,Durable,False,84.0,Shell fish


In [28]:
n = 5
pd_top_n = pd_dets.head(n)
pd_bot_n = pd_dets.tail(n)

In [46]:
highest = go.Bar(x = pd_top_n['ProductName'], y = pd_top_n['Quantity'], name = 'Best Sellers')
lowest = go.Bar(x = pd_bot_n['ProductName'], y = pd_bot_n['Quantity'], name = 'Worst Sellers')

layout = go.Layout(title = "Product performance: top and bottom {0}".format(n),
                  barmode='group')
iplot(go.Figure(data = [highest,lowest],layout = layout))